In [ ]:
# to automatically reload modules who's content has changed
%load_ext autoreload
%autoreload 2
# configure matplotlib
%matplotlib inline
#%config InlineBackend.figure_format = 'svg'

In [ ]:
from task_utils import *

In [ ]:
import funbo as fb
import funbo.plotting as fp

In [ ]:
def derivative(func):
    def df(X):
        dfs = []
        f = lambda x: np.asscalar(func(np.array([x])))
        for x in X:
            dfs.append(scipy.optimize.approx_fprime(x, f, epsilon=np.sqrt(np.finfo(float).eps)))
        return np.array(dfs).reshape(-1, 1)
    return df

class F:
    def __init__(self, f):
        self.f = f
        self.df = derivative(f)
        self.eval_count = 0
        
    def __call__(self, X, return_gradient=False):
        assert X.ndim == 2
        self.eval_count += X.shape[0]
        return (self.f(X), self.df(X)) if return_gradient else self.f(X)

class Trial:
    def __init__(self, f):
        np.random.seed(0)
        f.eval_count = 0
        self.f = f
        self.start = time.perf_counter()
    def finish(self, txs, tys):
        dur = time.perf_counter()-self.start
        print('{} evaluations in {:.2f}s'.format(self.f.eval_count, dur))
        fig, ax = plt.subplots(figsize=(12, 8))
        ax.plot(xs, f(xs.reshape(-1, 1)), label='f(x)')
        ax.scatter(txs.flatten(), tys.flatten(), label='test points')
        ax.legend()

In [ ]:
bounds = [(0, 10)]
xs = np.linspace(bounds[0][0], bounds[0][1], num=500)
f = F(lambda X: np.sin(X)*X + np.cos(X*3) * 0.1*X**2 - 1.5*X)

In [ ]:
def _():
    fig, ax = plt.subplots(figsize=(12, 8))
    fx, dfx = f(xs.reshape(-1, 1), return_gradient=True)
    ax.plot(xs, fx, label='f(x)')
    ax.plot(xs, dfx, '--', label='df_dx')
    ax.legend()
_()

In [ ]:
def _():
    t = Trial(f)
    n = 15
    txs, tys = fb.maximise_random(f, bounds, num_samples=n, num_take=n)
    t.finish(txs, tys)
_()

In [ ]:
def _():
    t = Trial(f)
    n = 1
    txs, tys = fb.maximise_quasi_Newton_restarts(f, bounds, num_its=n, num_take=n, exact_gradient=True)
    t.finish(txs, tys)
_()